# Shor algorithm

This notebook is part of a set of notebooks on different quantum algorithms. The main source of these notes is: (Vedral, 2013). 

## Theory

### Shor's algorithm

Shor's algorithm is a method for factoring a natural number $N$. It relies on the existance of an efficient classical algorithm for finding the greatest common factor (gcf) (called Euclid's algorithm (Lynn, 2000)) and on an efficient method to deduce some random eigenstates of an operator (called quantum phase estimation).

The algorithm is (based on (Vedral, 2013, p. 140)):

1. Randomly generate some natural number $a$ where $2<a<N$.

2. If $a$ and $N$ are not coprime $\left(K = gcf(a,N) \neq 1\right)$, then the factorization of $N$ is $\frac{N}{K},K$ and the algorithm stopped.

3. If $a$ is odd, then the algorithm fails. Otherwise, deduce the order of $a$ (smallest $r$ where $a^r \bmod N = 1$) by the phase estimation method (see below).

4. Then, as $a^r -1 \bmod N = (a^\frac{r}{2} - 1)(a^\frac{r}{2} + 1) \bmod N  = 0$ and as $a^\frac{r}{2} \bmod N  = 1$ can't be true (by the definition of $r$), if $N$ doesn't divide $b = a^\frac{r}{2} + 1$ then $a$ and $b$ share common factors. These can be deduced by Euclid's algorithm. Meaning if $K = gcd(a,b) \neq 1$ then the factorization of $N$ is $\frac{N}{K},K$.

It can be shown that this process has a success probability of $\frac{1}{2}$ (Vedral, 2013, p. 142) and the complexity of the algorithm can be shown to be $O((\log{n})^2(\log{\log{n}})(\log{\log{\log{n}}}))$ (Shor, 1996, p. 15).

### Euclid's algorithm

Euclid's algorithm finds the gcf of two natural numbers (here these numbers shall be denoted by $a,b$ and it will be assumed that $a<b$). The algorithm works by recursively applying the formula $gcd(a,b) = gcd(a,b-a)$ until the gcd is trivial (Lynn, 2000).

### Phase estimation

This is the quantum part of the algorithm; it efficiently finds the order $r$ associated with $a$ and $N$. To find $r$, a qubit system of dimension $m >\log_2(N)$ is set up with the normal binary labelling convention used for the basis (eg. $\ket{0} = \ket{000}, \ket{1} = \ket{010}, \ket{2} = \ket{011},...$). An operator $\hat{U}_a$ is set up where $\hat{U}_a\ket{x} = \ket{ax \bmod N}$ (for efficient implementation see (Shor, 1996, p. 11)). It can then be shown that the eigenstates of $\hat{U}_a$ take the form $\lambda_n = e^{\frac{2\pi i n}{r}}$ and that $\ket{1}$ is proportional to the sum of all eigenkets (Vendral, 2013, p. 144). Due to these properties, the quantum phase estimation algorithm can then be applied to $\ket{1}$ (see (Shor, 1996, p. 15) and (Vendral, 2013, p. 142) for a description and circuit diagrams). The result is an equally weighted superposition corresponding to each of the different eigenstates. A measurement can then be made to find one of these eigenstates. The resulting number is approximately $2^{2m+1}\frac{n}{r}$. $r$ can then be approximated by the denominator of the coprime fraction closest to $\frac{n}{r}$. This approximation can be made more accurate by performing the algorithm several times and taking the lowest common multiple (LCM) of the resulting r values.

## Implmentation

Below the algorithm is implemented. The quantum randomness is simulated by using a random number generator. Test cases were taken from: https://blendmaster.github.io/ShorJS/.



In [1]:
import numpy as np
import itertools
import sys
sys.path.append('C:\workspace\git-repos\physics-projects\quantum-algorithms')
%run register


Code for inverse QFT.

In [2]:
def invQFT(QFTRegister,start,end):
    Rs = []
    for i in range(2,end-start+1):
        Rs.append(np.array([[1,0],[0,np.exp(-2*np.pi*1j/(2**i),dtype=np.complex_)]],dtype=np.complex_))

    for i in reversed(range(end-start)):
        for j in reversed(range(end -start - i - 1)):
            gate = QFTRegister.findControlledSingleQubitOp(Rs[j], i+1+j, i)
            QFTRegister.applyGate(gate)

        QFTRegister.applyH(i)
    
    return QFTRegister


Test cases for inverse QFT.

In [3]:
#result should be 2
QFT1 = Register([q0 + complex(0,1)*q1,q0-q1,q0+q1])
print(f"Initial state: {QFT1.displayQubits()}")
print(f"Result: {invQFT(QFT1,0,3).displayQubits()}")

Initial state: (0.354+0j)|0> + (0.354+0j)|1> + (-0.354+0j)|2> + (-0.354+0j)|3> + 0.354j|4> + 0.354j|5> + -0.354j|6> + -0.354j|7>
Result: (1+0j)|2>


In [4]:
#result should be 1
QFT2 = Register([q0 + complex(0,1)*q1,q0-q1])
print(f"Initial state: {QFT2.displayQubits()}")
print(f"Result: {invQFT(QFT2,0,2).displayQubits()}")

Initial state: (0.5+0j)|0> + (-0.5+0j)|1> + 0.5j|2> + (-0-0.5j)|3>
Result: (1+0j)|1>


Code for phase estimation.

In [15]:
def findDominantBit(register, start, end):
    return int(register.binary[np.argmax(np.abs(register.qubits))][start:end],2)


def phaseEstimation(psiAcc,U,e):
    """
    Performs phase estimation.

    Inputs:
    psiAcc - accuracy of psi returned
    U - unitary matrix
    e - ket
    """

    register = Register([q0 for i in range(psiAcc)] + [e])
    print(f"Initial state (control qubits): {register.displayQubits(endBitsRemoved=1)}")

    register.applySingleQubitProduct(np.array([[1,1],[1,-1]])/np.sqrt(2),0,psiAcc)
    print(f"H applied (control qubits): {register.displayQubits(endBitsRemoved=1)}")
    
    for i in range(psiAcc): 
        gate = register.findControlledSingleQubitOp(np.linalg.matrix_power(U,2**i), psiAcc-1-i, psiAcc)
        register.applyGate(gate)

    print(f"H and Us applied (control qubits): {register.displayQubits(endBitsRemoved=1)}")

    #this is required as the QFT must be applied to the control qubits in reverse order
    register.invertQubits(0,register.n-1)

    register = invQFT(register,0,register.n-1)
    print(f"IQFT (control qubits): {register.displayQubits(endBitsRemoved=1)}")

    return register


Tests for phase estimation function.

In [18]:
#result should be 0
U1 = np.array([[1,0],[0,1]])
e1 = np.array([1,0],dtype=np.complex_) #I am doing the QFT on the last qubit
phaseEst1 = phaseEstimation(3,U1,e1)
bit1 = findDominantBit(phaseEst1,0,3)
print(f"Phase estimation result: {bit1/2**3}")

Initial state (control qubits): (1+0j)|0>
H applied (control qubits): (0.354+0j)|0> + (0.354+0j)|1> + (0.354+0j)|2> + (0.354+0j)|3> + (0.354+0j)|4> + (0.354+0j)|5> + (0.354+0j)|6> + (0.354+0j)|7>
H and Us applied (control qubits): (0.354+0j)|0> + (0.354+0j)|1> + (0.354+0j)|2> + (0.354+0j)|3> + (0.354+0j)|4> + (0.354+0j)|5> + (0.354+0j)|6> + (0.354+0j)|7>
IQFT (control qubits): (1+0j)|0>
Phase estimation result: 0.0


In [19]:
#result should be 0.25
U2 = np.array([[complex(0,1),0],[0,1]]) 
e2 = np.array([1,0],dtype=np.complex_)
phaseEst2 = phaseEstimation(2,U2,e2)
bit2 = findDominantBit(phaseEst2,0,2)
print(f"Phase estimation result: {bit2/2**2}")

Initial state (control qubits): (1+0j)|0>
H applied (control qubits): (0.5+0j)|0> + (0.5+0j)|1> + (0.5+0j)|2> + (0.5+0j)|3>
H and Us applied (control qubits): (0.5+0j)|0> + 0.5j|1> + (-0.5+0j)|2> + -0.5j|3>
IQFT (control qubits): (1+0j)|1>
Phase estimation result: 0.25


In [20]:
#result shoudl be 0.5 
U3 = np.array([[np.exp(np.pi*complex(0,1)),0],[0,1]],dtype=np.complex_) 
e3 = np.array([1,0],dtype=np.complex_)
phaseEst3 = phaseEstimation(8,U3,e3)
bit2 = findDominantBit(phaseEst3,0,8)
print(f"Phase estimation result: {bit2/2**8}")

Initial state (control qubits): (1+0j)|0>
H applied (control qubits): (0.062+0j)|0> + (0.062+0j)|1> + (0.062+0j)|2> + (0.062+0j)|3> + (0.062+0j)|4> + (0.062+0j)|5> + (0.062+0j)|6> + (0.062+0j)|7> + (0.062+0j)|8> + (0.062+0j)|9> + (0.062+0j)|10> + (0.062+0j)|11> + (0.062+0j)|12> + (0.062+0j)|13> + (0.062+0j)|14> + (0.062+0j)|15> + (0.062+0j)|16> + (0.062+0j)|17> + (0.062+0j)|18> + (0.062+0j)|19> + (0.062+0j)|20> + (0.062+0j)|21> + (0.062+0j)|22> + (0.062+0j)|23> + (0.062+0j)|24> + (0.062+0j)|25> + (0.062+0j)|26> + (0.062+0j)|27> + (0.062+0j)|28> + (0.062+0j)|29> + (0.062+0j)|30> + (0.062+0j)|31> + (0.062+0j)|32> + (0.062+0j)|33> + (0.062+0j)|34> + (0.062+0j)|35> + (0.062+0j)|36> + (0.062+0j)|37> + (0.062+0j)|38> + (0.062+0j)|39> + (0.062+0j)|40> + (0.062+0j)|41> + (0.062+0j)|42> + (0.062+0j)|43> + (0.062+0j)|44> + (0.062+0j)|45> + (0.062+0j)|46> + (0.062+0j)|47> + (0.062+0j)|48> + (0.062+0j)|49> + (0.062+0j)|50> + (0.062+0j)|51> + (0.062+0j)|52> + (0.062+0j)|53> + (0.062+0j)|54> + (0.06

Code for full Shor algorithm.

In [ ]:
#work in progress
def findNearestRational(x):
    """
    Function that finds the two coprime numbers whose fraction is nearest to x.
    """
    xb = np.round(x)
    xa = x - xb
    if xa != 0:
        xaRecpb = np.round(1/xa)
        return xb*xaRecpb+1,xaRecpb
    else:
        return xb,1
      

def shor(N,a,qNum,m,psiAcc):
    """
    Function that factorizes natural number N.

    Inputs:
    N - number being factorized
    a - int between 2 and N used in algorithm
    qNum - number of qubits used for U_a matrix
    m - number of runs of phase estimation algorithm
    psiAcc - accuracy of psi found in phase estimation
    """
    HDim = 2**qNum

    K = np.gcd(a,N)
    if K != 1 and K != N:
        return N/K,K
    
    #find U_a matrix
    U_a = np.zeros([HDim,HDim],dtype=np.complex_)
    for j,k in itertools.product(range(HDim), repeat=2): 
        if j == a*k % N and k < N:
            U_a[j,k] = 1
        elif j == k and k >= N:
            U_a[j,k] = 1
            
    #random number generator models quantum probablities
    eigenVal, eigenVec = np.linalg.eig(U_a)
    
    rs = []
    for l in range(m):
        index = np.random.randint(0,high=HDim)
        phaseEst = phaseEstimation(psiAcc,U_a,eigenVec[index])
        if phaseEst != None:
            kj,rj = findNearestRational(phaseEst)
            rs.append(rj)
    
    if rs != []:
        R = np.lcm.reduce(np.array(rs,dtype=int))

        if R % 2 == 0:
            g = np.gcd(int(a**(R/2) + 1),int(N)) 
            if g != 1 and g != N:
                return N/g, g
    
    return "failed"


Tests for Shor algorithm.

In [19]:
print(f"12 is factorized to: {shor(12,5,4,6,4)}") 

Initial state: (0.707+0j)|0> + (-0.707+0j)|1>


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 256 is different from 32)

In [ ]:
#should fail
print(f"21 is factorized to: {shor(21,5,4,6,4)}") 

21 is factorized to: failed


# References

Vedral, V., 2013. Introduction to Quantum Information Science. Oxford: Oxford University Press.

Lynn, B., 2000. Euclid's Algorithm. [Online] 
Available at: https://crypto.stanford.edu/pbc/notes/numbertheory/euclid.html
[Accessed 27 August 2024].

Shor, P., 1996. Polynomial-Time Algorithms for Prime Factorization and Discrete Logarithms on a Quantum Computer. 



